In [1]:
%env CUDA_LAUNCH_BLOCKING=1
import torch
from torch import nn
from torch.utils.data import DataLoader
from data_synth import ORDataset, ANDDataset, DNFDataset
from fuzzy_logic import FuzzyOR, FuzzyAND, FuzzyDNF
from ideal_logic import ORIdealModel, ANDIdealModel
from plot import plot_loss, plot_bit_density

env: CUDA_LAUNCH_BLOCKING=1


In [2]:
%%capture
from tqdm.notebook import tqdm
tqdm().pandas()

In [38]:
def train_dnf(model, dataloader, lr):
    optimizer = torch.optim.Adam(model.parameters(), lr)
    loss_fn = torch.nn.CrossEntropyLoss()

    seen = 0
    items = []
    obs_losses = []

    for batch, (X, y) in tqdm(enumerate(dataloader), total=len(dataloader)):
        X = X.to('cuda')
        y = y.to('cuda')
        pred = torch.squeeze(model(X))
        print(pred.shape, y.shape)
        loss = loss_fn(pred, y)

        # Optimise NN model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        model.harden()

        seen += len(X)
        items.append(seen)
        obs_losses.append(loss.item())
        
    return torch.tensor(items), torch.tensor(obs_losses)

In [39]:
dnf_dim = 5
dnf_dataset = DNFDataset(dnf_dim, 20000, 0.5, 3, 0.3, 0.3)
dnf_dataloader = DataLoader(dnf_dataset, batch_size=128, shuffle=True)

dnf_model = FuzzyDNF(dnf_dim, 1, 3)
dnf_model.to('cuda')

dnf_items, dnf_losses = train_dnf(dnf_model, dnf_dataloader, lr=1e-1)

plot_loss(dnf_items, dnf_losses, ylabel='Observed Loss', approx=True)
print(dnf_dataset.conj_sets)
print(dnf_dataset.disj_set)
print(dnf_dataset.X)

  0%|          | 0/157 [00:00<?, ?it/s]

torch.Size([128]) torch.Size([128])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)